In [1]:
# scrape dr.dk with selenium get titles of all news articles

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd

from webdriver_manager.chrome import ChromeDriverManager

In [3]:
import pandas as pd
import numpy as np

df_dates = pd.read_excel('TaylorSwiftCross.xlsx')
citites = list(df_dates.Cities.unique())
df_dates

,Cities,Day_pre,Day_after,Treat
0,Stockholm,2024-05-17,2024-05-20,1
1,Paris,2024-05-09,2024-05-12,1
2,Amsterdam,2024-07-04,2024-07-06,1
3,Lisbon,2024-05-24,2024-05-25,1
4,Warsaw,2024-08-01,2024-08-03,1
5,Milan,2024-07-13,2024-07-14,1
6,Zürich,2024-07-09,2024-07-10,1
7,Munchen,2024-07-27,2024-07-28,1
8,Madrid,2024-05-29,2024-05-31,1
9,Lyon,2024-06-02,2024-06-03,1


In [5]:
# open chromedriver
driver = webdriver.Chrome()
driver.get("https://worldcitiesranking.com/detailed-list-150-cities/")

In [8]:
col_names = ["City", "Sights", "Location", "Pedestrian Zones", "Affordability", "Safety", "Cleanliness", "Shopping", "Climate"]

data = {}
for i, col in enumerate(col_names):
    l_now = []
    
    elems = driver.find_elements(By.XPATH, f"/html/body/div/div[2]/article/div/div[2]/div/table/tbody/tr/td[{i+2}]")
                                        #"/html/body/div/div[2]/article/div/div[2]/div/table/tbody/tr[145]/td[2]
    for e in elems:
        l_now.append(e.text)

    data[col] = l_now

df_cities = pd.DataFrame(data)

In [9]:
df_cities

,City,Sights,Location,Pedestrian Zones,Affordability,Safety,Cleanliness,Shopping,Climate
0,Paris,9.20,6.50,6.50,1.50,6.75,6.25,8.50,6.00
1,Venice,8.35,9.75,9.75,1.50,7.50,6.75,3.25,6.50
2,Rome,8.95,6.50,6.50,3.25,6.50,6.50,7.25,6.50
3,Barcelona,7.20,8.00,7.50,3.25,6.75,6.00,7.50,9.00
4,London,8.85,6.25,6.00,1.25,7.00,7.00,8.00,5.75
...,...,...,...,...,...,...,...,...,...
145,Antalya,5.35,7.75,6.00,5.00,5.75,6.00,5.00,7.50
146,Montevideo,5.50,8.50,5.50,5.50,5.00,6.00,5.25,7.00
147,"Washington, D.C.",7.00,6.50,4.50,3.25,5.75,6.75,5.75,6.75
148,Glasgow,6.25,5.75,6.25,3.25,6.75,6.75,5.50,5.25


In [10]:
cities = list(df_dates['Cities'].unique())

data_out = {}
data_out['City'] = []
data_out['Sights'] = []
data_out['Safety'] = []
data_out['Shopping'] = []
data_out['Climate'] = []

dfs = pd.DataFrame()
for city in cities:

    I = df_cities['City'] == city
    df_temp = df_cities[I]

    if len(df_temp) == 0:
        continue

    dfs = pd.concat([dfs, df_temp[['City', 'Sights', 'Safety', 'Shopping', 'Location', 'Climate', 'Affordability']]])
            

In [12]:
df_out = dfs.reset_index()
df_out.drop(columns=['index'], inplace=True)
df_out.to_csv('city_rankings.csv', index=False)

In [13]:
df_out

,index,City,Sights,Safety,Shopping,Location,Climate,Affordability
0,25,Stockholm,6.75,8.75,6.25,9.00,5.00,1.50
1,0,Paris,9.20,6.75,8.50,6.50,6.00,1.50
2,18,Amsterdam,7.30,7.50,6.50,8.75,5.50,2.50
3,19,Lisbon,7.40,7.50,6.00,7.75,6.50,3.75
4,67,Warsaw,6.50,7.00,5.75,5.75,5.75,5.00
5,35,Milan,7.35,6.75,7.50,5.50,5.25,3.00
6,132,Zürich,5.65,8.75,5.00,8.25,4.25,0.75
7,10,Madrid,7.50,7.00,7.50,5.25,8.75,3.75
8,50,Lyon,6.25,6.75,5.50,7.25,6.75,3.50
9,59,Hamburg,6.40,7.00,6.00,8.25,5.75,2.25
